In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 4.2MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bioni

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
#driver 실행
def drive(url):
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options) #driver 객체 불러옴
    driver.implicitly_wait(3) # 3초 후에 작동하도록
    driver.get(url) #url에 접속
    html = driver.page_source #현재 driver에 나타난 창의 page_source(html) 가져오기
    soup = BeautifulSoup(html, 'html.parser') #html 파싱(parsing)을 위해 BeautifulSoup에 넘겨주기
    return driver, soup

In [ ]:
#  class 명이 title 인 a 테그를 가져와 모든 요일 별 웹툰의 제목과 링크만 추출해낸다.
def webtoon_link_list(soup):
  class_title_a_list = soup.select("a.title")
  title_list = []
  for a in class_title_a_list:
    a_href = a.get('href') # 테그 안의 내용을 가져온다(인자로 'href'속성을 사용했다)
    a_text = a.get_text() #
    result = (a_text,f'https://comic.naver.com/{a_href}')
    title_list.append(result)
  return title_list

In [ ]:
# soup 테이블 테그로 부터 해당 웹툰 의 회차 리스트를 가져오는 함수
def get_link(soup):
  table_list = soup.select("td.title")
  episode_list = []
  for table in table_list:
    link = table.select("a")[0].get('href')
    episode_list.append(f'https://comic.naver.com{link}')
  return episode_list

In [ ]:
# 웹툰 모든 회차의 링크를 가져오는 함수
def get_link_list(title_list):
  link_list=[]
  # 웹툰 리스트에서 웹툰 하나당 처리
  for title in title_list:
    webtoon = requests.get(title[1])
    soup = BeautifulSoup(webtoon.text, 'html.parser')
    # 해당 웹툰의 첫 페이지에 실린 회차들 링크를 가져옴
    link_list += get_link(soup)
  return link_list

In [ ]:
# 한 에피소드에서 댓글을 크롤링 하는 함수
def get_comment_link(link):
  titleId = link.split('titleId=')[1].split('&')[0]
  no = link.split('no=')[1].split('&')[0]
  return f'https://comic.naver.com/comment/comment.nhn?titleId={titleId}&no={no}'

In [ ]:
# 댓글 iframe html에서 댓글 리스트 반환하는 함수
def get_comments(driver,link):
  comments = []
  driver.get(link)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  raw = soup.select('.u_cbox_contents')
  for r in raw:
    comments.append(r.text)
  return comments

In [ ]:
# 댓글 크롤링 함수를 이용해 link list로 부터 모든 댓글을 크롤링 해온다.
def comments_all_episode(link_list):
  base_link='https://comic.naver.com/webtoon/weekday.nhn'
  driver,_ = drive(base_link)
  comments_list=[]
  for episode in link_list:
    link = get_comment_link(episode)
    comments_list += get_comments(driver,link)
  return comments_list

In [ ]:
# 데이터를 섞고 csv로 저장하는 함수
def save_data(data):
  import random
  import pandas as pd
  random.shuffle(cmts)
  label = [0 for c in cmts]
  df = pd.DataFrame(list(zip(cmts,label)),columns=['comment','label'] )
  df.to_csv('webtoon_comments_unlabeled.csv',sep=',',encoding='utf-16')

In [ ]:
# 1.네이버 comics 메인 페이지로부터 html 문서를 긁어온다.(beutifulsoup를 이용)
URL_TOTAL_LIST = 'https://comic.naver.com/webtoon/weekday.nhn'
response = requests.get(URL_TOTAL_LIST)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# 2.만화 링크 리스트를 크롤링
title_list = webtoon_link_list(soup)

In [ ]:
# 3.모든 만화의 최신 1페이지에 있는 회차 링크 리스트를 크롤링
link_list = get_link_list(title_list)

In [ ]:
# 4.모든 만화 모든 회차의 댓글을 크롤링
cmts = comments_all_episode(link_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 중복 댓글 제거
cmts = set(cmts)

In [ ]:
# 데이터를 csv로 저장
cmts=list(cmts)
save_data(cmts)

In [ ]:
# csv 파일 불러오기
import pandas as pd

In [ ]:
# 데이터가 올바르게 저장됐는 지 확인
df = pd.read_csv('webtoon_comments_unlabeled.csv',encoding='utf-16')

In [ ]:
df.head()

,Unnamed: 0,comment,label
0,0,와 목숨 보장한다는 약속을 절대 깨지 못한다고?? 그러면 박윤수는 스스로 채아의 올...,0
1,1,희준이 여친이 푹빠질만 하네ㄷㄷ상남자네,0
2,2,화이팅!그리고 저는 이제 안볼려구요^^,0
3,3,결혼이라는 제도로 1남성에 1여성을 보급하는 비정상적인 가부장제를 벗어난 세계에서도...,0
4,4,제발 작가님아 미오후 뇌절좀 그만해요... 지금까지 미오후로 만화 쳐망가뜨려놓고 ...,0
